In [1]:
import pandas as pd # Importa biblioteca Pandas

In [2]:
# Importa as faturas SP e Faturas MG
sp = pd.read_excel(r'N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\02_Faturas\Faturas_New.xlsx', sheet_name='Mestre')
mg = pd.read_excel(r'N:\Drives compartilhados\04 - OPERAÇÕES\70. Transporte\02_Faturas\Faturas MG.xlsx', sheet_name = 'Mestre')

In [3]:
# Dropa as colunas não utilizaveis dos 2 Dataframes
dropa_sp = ['Fatura', 'Resp','Data de Emissão',	'Comentários',	'Data de Verif',	'Diferença %', 'Status'	] #'Unnamed: 14', 'Unnamed: 15, 'Data de Emissão''
dropa_mg = ['Fatura', 'Resp', 'Data de Emissão','Coluna1', 'data de conferência',	'Diferença %',	'Status']

In [4]:
# Dropa as colunas não utilizaveis dos 2 Dataframes
sp = sp.drop(columns = dropa_sp)
mg = mg.drop(columns = dropa_mg)


In [5]:
# Cria a coluna Origem para identificar de onde a Nota Fiscal eh oriunda. 
sp['Origem'] = 'Faturas SP'
mg['Origem'] = 'Faturas MG'

In [ ]:
#Concatena os 2 dataframes em um só.
faturas = pd.concat([mg, sp], ignore_index=True)
faturas

In [7]:
# Função que indetifica as Taxas de Zona Rural e demais taxas.
def identifica(motivo):
    taxas = ["TDE", "TZR", "TDA"]
    return motivo in taxas

In [ ]:
# Criação das colunas referente a 'Promo Frete' e 'Taxas'.
faturas["Promo"] = faturas.apply(lambda row: row["Cobrado"] if row["Motivo"] == "Promo Frete" else None, axis=1)
faturas["Taxas"] = faturas.apply(lambda row: row["Cobrado"] if identifica(row["Motivo"]) else None, axis=1) # Chama a função Indentifica para encontrar as taxas.
faturas

In [ ]:
# Agrupa todas as notas fiscais do dataframe para não ter Relacionamento de (*:*) com a base Mestre.
soma = ['Transportadora', 'Cobrado', 'Recebido ','Diferença', 'Motivo', 'Promo', 'Novo Cobrado','Taxas', 'Origem' ]
faturas = faturas.groupby('NF')[soma].sum().reset_index()
faturas

In [ ]:
# Tratamento preguiçoso da coluna Origem.
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MGFaturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas MGFaturas MGFaturas MGFaturas MGFaturas MG', 'Faturas MG', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SP', 'Faturas SP', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SPFaturas SP', 'Faturas SP', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SPFaturas SPFaturas SP', 'Faturas SP', case=False)
faturas['Origem'] = faturas['Origem'].str.replace('Faturas SPFaturas SPFaturas SPFaturas SPFaturas SP', 'Faturas SP', case=False)
faturas

In [ ]:
faturas = faturas.iloc[:-4]
faturas

In [13]:
#Salva o arquivo
#faturas.to_excel(r'N:\Drives compartilhados\22 - BI\BI Operações\Logística\Custo de Frete\Faturas.xlsx', index = False) # Salva Faturas para o Bi tradicional
faturas.to_excel(r'N:\Drives compartilhados\28 - Dados BI\2 Dados Transformados\2.Frete\2.Frete_Transportadora.xlsx', index = False) # Salva Faturas para o Data Product